In [ ]:
# Data Processing from base data to dataset for Truth-in-Tax vs Statutory rate counterfactual

In [ ]:
import pandas as pd

df_clean = pd.read_json("../../public/data/base_data.json")

# drop needless columns